# MadMiner example

In this tutorial we'll demonstrate how to use MadMiner to generate train and test samples for the ML methods introduced in ["Constraining Effective Field Theories With Machine Learning"](https://arxiv.org/abs/1805.00013) and ["A Guide to Constraining Effective Field Theories With Machine Learning"](https://arxiv.org/abs/1805.00020), both by Johann Brehmer, Gilles Louppe, Juan Pavez, and Kyle Cranmer.

Before you execute this notebook, make sure you have running installations of MadGraph, Pythia, and Delphes. Note that at least for now, the MG-Pythia interface and Delphes require custom patches (available upon request). In addition, MadMiner and [DelphesMiner](https://github.com/johannbrehmer/delphesminer) have to be in your PYTHONPATH.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
from matplotlib import pyplot as plt
% matplotlib inline

from madminer.goldmine import GoldMine
from madminer.tools.plots import plot_2d_morphing_basis

from delphesprocessor.delphesprocessor import DelphesProcessor

from madminer.refinery import combine_and_shuffle
from madminer.refinery import Refinery
from madminer.refinery import constant_benchmark_theta, multiple_benchmark_thetas
from madminer.refinery import constant_morphing_theta, multiple_morphing_thetas, random_morphing_thetas

from forge.train import train

Please enter here the path to your MG5 root directory. This notebook assumes that you installed Delphes and Pythia through MG5.

In [ ]:
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2'

## 1. Define parameter space

After creating a `GoldMine` instance, the first important step is the definition of the parameter space. Each model parameter is characterized by a name as well as the LHA block and ID.

If morphing is used, one also has to specify the maximal power with which the parameter contributes to the squared matrix element. For instance, a parameter that contributes only to one vertex, will typically have `morphing_max_power=2`, while a parameter that contributes to two vertices usually has `morphing_max_power=4`. Exceptions arise for instance when the interference effects between the SM and dimension-six operators are modelled, but the square of the dimension-six amplitude (subleading in 1/Lambda) is not taken into account, in which case `morphing_max_power=1`. Finally, the `parameter_range` argument defines the range of parameter values that are used for the automatic optimization of the morphing basis.

In [ ]:
miner = GoldMine()

miner.add_parameter(
    lha_block='dim6',
    lha_id=2,
    parameter_name='CWL2',
    morphing_max_power=2,
    parameter_range=(-10.,10.)
)
miner.add_parameter(
    lha_block='dim6',
    lha_id=5,
    parameter_name='CPWL2',
    morphing_max_power=2,
    parameter_range=(-10.,10.)
)

## 2. Define benchmark points (evaluation points for |M|^2)

The next step is the definition of all the points at which the weights (squared matrix elements) should be evaluated by MadGraph. We call these points "benchmarks".

### 2a. Set benchmarks by hand

One can define benchmarks by hand:

In [ ]:
miner.add_benchmark(
    {'CWL2':0., 'CPWL2':0.},
    'sm'
)
miner.add_benchmark(
    {'CWL2':1., 'CPWL2':0.},
    'bsm'
)

### 2b. Benchmarks for morphing

If morphing is used, the function `set_benchmarks_from_morphing` has to be called. With the option `keep_existing_benchmarks=True`, MadMiner will keep all the benchmark points defined beforehand and run a simple optimization algorithm to fix the remaining ones for the basis (which may be none). Otherwise, MadMiner will optimize the full basis and forget about all previously defined benchmark points. The argument `n_trials` determines the number of random candidate bases that the optimization algorithm goes through.

In [ ]:
miner.set_benchmarks_from_morphing(
    keep_existing_benchmarks=True,
    n_trials=1000,
    max_overall_power=2
)

Let's have a look at the resulting morphing basis and the "morphing error", i.e. the sum of squared morphing weights as a function of the parameter space:

In [ ]:
fig = plot_2d_morphing_basis(
    miner.morpher,
    xlabel=r'$c_{W} / \Lambda^2$ [TeV$^{-2}$]',
    ylabel=r'$c_{\tilde{W}} / \Lambda^2$ [TeV$^{-2}$]',
    xrange=(-10.,10),
    yrange=(-10.,10.)
)

## 3. Save settings and run MadGraph

The parameter space, benchmark points, and morphing setup are saved in a HDF5 file:

In [ ]:
miner.save('data/madminer_example.h5')

In a next step, MadMiner starts MadGraph and Pythia to generate events and calculate the weights. You have to provide paths to the process card, run card, param card (the entries corresponding to the parameters of interest will be automatically adapted), and an empty reweight card.

The `sample_benchmark` option can be used to specify which benchmark should be used for sampling. If it is not used, MadMiner will automatically use the benchmark that was added first. Finally, if MadGraph is supposed to run in a different Python environment or requires other setup steps, you can use the `initial_command` argument.

In [ ]:
miner.run(
    mg_directory=mg_dir,
    proc_card_file='cards/proc_card.dat',
    param_card_template_file='cards/param_card_template.dat',
    reweight_card_template_file='cards/reweight_card_template.dat',
    run_card_file='cards/run_card.dat',
    pythia8_card_file='cards/pythia8_card.dat',
    sample_benchmark='sm',
    initial_command='source activate python2'
)

## 4. Run detector simulation and extract observables

The detector simulation and calculation of observables is not part of MadMiner. The reason is that different users might have very different requirements here: while a phenomenologist might be content with the fast detector simulation from Delphes, an experimental analysis might require the full simulation through Geant4.

We provide the DelphesMiner package, which wraps around Delphes and allows for the fast extraction of observables into the HDF5 file.

Any user is free to replace the DelphesMiner step with a tool of their choice. 

In [ ]:
dm = DelphesProcessor()

After creating the DelphesProcessor object, one can add a number of HepMC event samples...

In [ ]:
dm.add_hepmc_sample('MG_process/Events/run_01/tag_1_pythia8_events.hepmc.gz')

... and have it run Delphes:

In [ ]:
dm.run_delphes(delphes_directory=mg_dir + '/Delphes',
               delphes_card='cards/delphes_card.dat',
               initial_command='source activate python2')

The next step is the definition of observables through a name and a python expression. For the latter, you can use the objects `j[i]`, `e[i]`, `mu[i]`, `a[i]`, `met`, where the indices `i` refer to a ordering by the transverse momentum. All of these objects are scikit-hep [LorentzVectors](http://scikit-hep.org/api/math.html#vector-classes), see the link for a documentation of their properties.

There is an optional keyword `required`. If `required=True`, we will only keep events where the observable can be parsed, i.e. all involved particles have been detected. If `required=False`, un-parseable observables will be filled with `np.nan`.

In [ ]:
dm.add_observable('pt_j1', 'j[0].pt', required=True)
dm.add_observable('pt_j2', 'j[1].pt', required=True)
dm.add_observable('delta_phi_jj', 'abs(j[0].phi() - j[1].phi())', required=True)
dm.add_observable('delta_eta_jj', 'abs(j[0].eta - j[1].eta)', required=True)
dm.add_observable('m_jj', '(j[0] + j[1]).m', required=True)
dm.add_observable('n_jets', 'len(j)', required=True)

The function `analyse_delphes_samples` extracts all these observables from the Delphes ROOT file(s) generated before:

In [ ]:
dm.analyse_delphes_samples()

The values of the observables and the weights are then saved in the HDF5 file. It is possible to overwrite the same file, or to leave the original file intact and save all the data into a new file as follows:

In [ ]:
dm.save('data/madminer_example_with_data.h5', 'data/madminer_example.h5')

It's easy to check some distributions at this stage

In [ ]:
fig = plt.figure(figsize=(5,5))

for weights in dm.weights:
    plt.hist(dm.observations['pt_j1'], range=(0.,800.), bins=20, histtype='step', weights=weights)

plt.show()

## 5. Combine and shuffle different event samples

To reduce disk usage, you can generate several small event samples with the steps given above, and combine them now. Note that (for now) it is essential that all of them are generated with the same setup, including the same benchmark points / morphing basis!

In [ ]:
combine_and_shuffle(
    ['data/madminer_example_with_data.h5'],
    'data/madminer_example_shuffled.h5'
)

## 6. Make (unweighted) training and test samples

The last important MadMiner class is the `Smithy`. From all the data we have in the HDF5 file now, it extracts unweighted samples including the augmented data ("gold") that is needed as training and evaluation data for the Machine Learning algorithms.

In [2]:
refinery = Refinery('data/madminer_example_shuffled.h5', debug=True)

15:09  
15:09  ------------------------------------------------------------
15:09  |                                                          |
15:09  |  MadMiner                                                |
15:09  |                                                          |
15:09  |  Version from July 31, 2018                              |
15:09  |                                                          |
15:09  |           Johann Brehmer, Kyle Cranmer, and Felix Kling  |
15:09  |                                                          |
15:09  ------------------------------------------------------------
15:09  
15:09  Loading data from data/madminer_example_shuffled.h5
15:09  Found 2 parameters:
15:09     CWL2 (LHA: dim6 2, maximal power in squared ME: 2, range: (-10.0, 10.0))
15:09     CPWL2 (LHA: dim6 5, maximal power in squared ME: 2, range: (-10.0, 10.0))
15:09  Found 6 benchmarks:
15:09     sm: CWL2 = 0.00e+00, CPWL2 = 0.00e+00
15:09     bsm: CWL2 = 1.00, CPWL2 = 0.00e+00

The `Refinery` class defines four different high-level functions to generate train or test samples:
- `extract_samples_train_plain()`, which only saves observations x, for instance for histograms or ABC;
- `extract_samples_train_local()` for methods like SALLY and SALLINO;
- `extract_samples_train_ratio()` for techniques like CARL, ROLR, CASCAL, and RASCAL; and
- `extract_samples_test()` for the evaluation of any method.

For the arguments `theta`, `theta0`, or `theta1`, you can use the helper functions `constant_benchmark_theta()`, `multiple_benchmark_thetas()`, `constant_morphing_theta()`, `multiple_morphing_thetas()`, and `random_morphing_thetas()`, all defined in the `smithy` module.

In [ ]:
x, theta, t_xz = refinery.extract_samples_train_local(
    theta=constant_morphing_theta(np.array([1.,0.])),
    n_samples=1000,
    folder='./data/samples',
    filename='train_sally'
)

In [ ]:
x, theta0, theta1, y, r_xz, t_xz = refinery.extract_samples_train_ratio(
    theta0=random_morphing_thetas(None, [('gaussian', 0., 5.), ('flat', -10., 10.)]),
    theta1=constant_benchmark_theta('sm'),
    n_samples=10000,
    folder='./data/samples',
    filename='train_rascal'
)

15:09  Extracting training sample for ratio-based methods. Numerator hypothesis: ('random', (None, [('gaussian', 0.0, 5.0), ('flat', -10.0, 10.0)])), denominator hypothesis: ('benchmark', 'sm')
15:09  Total n_samples: 5000, n_benchmarks: 5000, n_samples_per_theta: 1
15:09  Augmented data requested:
15:09    Joint ratio, num sampling None, den auxiliary None
15:09    Joint score, at sampling None
15:09  Sampling and auxiliary thetas before balancing:
15:09    sampling thetas:  5000 types, 5000 values
15:09    auxiliary thetas: 1 types, 1 values
15:09  Samples per theta: 1
15:09  Samples per theta: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

15:09  Sampling and auxiliary thetas after balancing:
15:09    sampling thetas:  5000 types, 5000 values
15:09    auxiliary thetas: 5000 types, 5000 values
15:09  New theta configuration
15:09    Sampling theta: morphing, [5.3591559  5.43841406]
15:09    Auxiliary theta: benchmark, sm
15:09    # samples: 1
15:09  xsec for this sampling theta: (0.26475054178143653 +/- 0.023334255223519514) pb
15:09  Cumulative probability (should be close to 1): 1.0001019135142681
15:09  Cumulative x shape: (1, 1, 6)
15:09  New theta configuration
15:09    Sampling theta: morphing, [-1.9947305  -7.00961003]
15:09    Auxiliary theta: benchmark, sm
15:09    # samples: 1
15:09  xsec for this sampling theta: (0.30970368983226026 +/- 0.006983642712242623) pb
15:09  Cumulative probability (should be close to 1): 0.9999787456044029
15:09  Cumulative x shape: (2, 1, 6)
15:09  New theta configuration
15:09    Sampling theta: morphing, [-6.63968714  1.74009612]
15:09    Auxiliary theta: benchmark, sm
15:09    # s

15:09  Cumulative probability (should be close to 1): 0.9999837862673747
15:09  Cumulative x shape: (23, 1, 6)
15:09  New theta configuration
15:09    Sampling theta: morphing, [ 4.52036514 -6.1450919 ]
15:09    Auxiliary theta: benchmark, sm
15:09    # samples: 1
15:09  xsec for this sampling theta: (0.2690667402281657 +/- 0.010021102703236915) pb
15:09  Cumulative probability (should be close to 1): 1.0000369414829884
15:09  Cumulative x shape: (24, 1, 6)
15:09  New theta configuration
15:09    Sampling theta: morphing, [-3.75732112  4.25320651]
15:09    Auxiliary theta: benchmark, sm
15:09    # samples: 1
15:09  xsec for this sampling theta: (0.3173101744491474 +/- 0.004992542161296278) pb
15:09  Cumulative probability (should be close to 1): 0.9999779752609517
15:09  Cumulative x shape: (25, 1, 6)
15:09  New theta configuration
15:09    Sampling theta: morphing, [-5.43169166 -2.42009979]
15:09    Auxiliary theta: benchmark, sm
15:09    # samples: 1
15:09  xsec for this sampling the

15:09  xsec for this sampling theta: (0.29049398999614034 +/- 0.009811231347955566) pb
15:09  Cumulative probability (should be close to 1): 0.9999905212326269
15:09  Cumulative x shape: (46, 1, 6)
15:09  New theta configuration
15:09    Sampling theta: morphing, [ 6.59706937 -2.96611927]
15:09    Auxiliary theta: benchmark, sm
15:09    # samples: 1
15:09  xsec for this sampling theta: (0.25461208009812386 +/- 0.019936854776031635) pb
15:09  Cumulative probability (should be close to 1): 1.0001019517982042
15:09  Cumulative x shape: (47, 1, 6)
15:09  New theta configuration
15:09    Sampling theta: morphing, [-3.96086546  2.92993887]
15:09    Auxiliary theta: benchmark, sm
15:09    # samples: 1
15:09  xsec for this sampling theta: (0.31664340816569636 +/- 0.004662723738854316) pb
15:09  Cumulative probability (should be close to 1): 0.9999788528485531
15:09  Cumulative x shape: (48, 1, 6)
15:09  New theta configuration
15:09    Sampling theta: morphing, [-4.90030971  4.86050244]
15:09 

15:09  Cumulative probability (should be close to 1): 1.0002690986535383
15:09  Cumulative x shape: (68, 1, 6)
15:09  New theta configuration
15:09    Sampling theta: morphing, [4.40763317 3.05778162]
15:09    Auxiliary theta: benchmark, sm
15:09    # samples: 1
15:09  xsec for this sampling theta: (0.2638194949830476 +/- 0.015848866158672058) pb
15:09  Cumulative probability (should be close to 1): 1.0000698340924525
15:09  Cumulative x shape: (69, 1, 6)
15:09  New theta configuration
15:09    Sampling theta: morphing, [ 6.95075174 -9.50284848]
15:09    Auxiliary theta: benchmark, sm
15:09    # samples: 1
15:09  xsec for this sampling theta: (0.2717683082216704 +/- 0.019098949734717326) pb
15:09  Cumulative probability (should be close to 1): 1.0000708713788284
15:09  Cumulative x shape: (70, 1, 6)
15:09  New theta configuration
15:09    Sampling theta: morphing, [ 2.84856776 -1.92287079]
15:09    Auxiliary theta: benchmark, sm
15:09    # samples: 1
15:09  xsec for this sampling theta

...

15:10  Warning: large statistical uncertainty on the total cross section for theta = [ 9.04683186 -3.97159832]: (0.24923201559356095 +/- 0.032829815781088414) pb
15:10  Cumulative probability (should be close to 1): 1.0001740094615459
15:10  Cumulative x shape: (1837, 1, 6)
15:10  New theta configuration
15:10    Sampling theta: morphing, [5.60710029 0.90366368]
15:10    Auxiliary theta: benchmark, sm
15:10    # samples: 1
15:10  xsec for this sampling theta: (0.25669258795224786 +/- 0.019496789020990835) pb
15:10  Cumulative probability (should be close to 1): 1.0000931509286841
15:10  Cumulative x shape: (1838, 1, 6)
15:10  New theta configuration
15:10    Sampling theta: morphing, [5.88800711 1.52721547]
15:10    Auxiliary theta: benchmark, sm
15:10    # samples: 1
15:10  xsec for this sampling theta: (0.2560386411106337 +/- 0.021772017716936615) pb
15:10  Cumulative probability (should be close to 1): 1.0001036091655495
15:10  Cumulative x shape: (1839, 1, 6)
15:10  New theta confi

15:10    Sampling theta: morphing, [ 7.61269116 -5.65760546]
15:10    Auxiliary theta: benchmark, sm
15:10    # samples: 1
15:10  xsec for this sampling theta: (0.2565565011984512 +/- 0.022395906883164337) pb
15:10  Cumulative probability (should be close to 1): 1.00011453652872
15:10  Cumulative x shape: (1859, 1, 6)
15:10  New theta configuration
15:10    Sampling theta: morphing, [-4.46290116 -7.485557  ]
15:10    Auxiliary theta: benchmark, sm
15:10    # samples: 1
15:10  xsec for this sampling theta: (0.33079925706435304 +/- 0.004097266875112519) pb
15:10  Cumulative probability (should be close to 1): 0.9999871613739657
15:10  Cumulative x shape: (1860, 1, 6)
15:10  New theta configuration
15:10    Sampling theta: morphing, [1.8411519  8.24459568]
15:10    Auxiliary theta: benchmark, sm
15:10    # samples: 1
15:10  xsec for this sampling theta: (0.29073986089026677 +/- 0.007982883474289396) pb
15:10  Cumulative probability (should be close to 1): 1.0000271698137053
15:10  Cumulat

15:10    # samples: 1
15:10  xsec for this sampling theta: (0.318601129978082 +/- 0.0039936377942948464) pb
15:10  Cumulative probability (should be close to 1): 0.9999816841018732
15:10  Cumulative x shape: (1882, 1, 6)
15:10  New theta configuration
15:10    Sampling theta: morphing, [2.00368877 8.82572708]
15:10    Auxiliary theta: benchmark, sm
15:10    # samples: 1
15:10  xsec for this sampling theta: (0.29219414313163644 +/- 0.00886685504645383) pb
15:10  Cumulative probability (should be close to 1): 1.0000307789876044
15:10  Cumulative x shape: (1883, 1, 6)
15:10  New theta configuration
15:10    Sampling theta: morphing, [ 3.27760797 -8.67977753]
15:10    Auxiliary theta: benchmark, sm
15:10    # samples: 1
15:10  xsec for this sampling theta: (0.28330051190609856 +/- 0.01131249525099729) pb
15:10  Cumulative probability (should be close to 1): 1.0000073142152623
15:10  Cumulative x shape: (1884, 1, 6)
15:10  New theta configuration
15:10    Sampling theta: morphing, [-0.98655

15:10    Auxiliary theta: benchmark, sm
15:10    # samples: 1
15:10  xsec for this sampling theta: (0.2545048578092069 +/- 0.03615013629178525) pb
15:10  Warning: large statistical uncertainty on the total cross section for theta = [7.59030025 4.36183321]: (0.2545048578092069 +/- 0.03615013629178525) pb
15:10  Cumulative probability (should be close to 1): 1.0001700639079443
15:10  Cumulative x shape: (1904, 1, 6)
15:10  New theta configuration
15:10    Sampling theta: morphing, [0.8983032  3.17212427]
15:10    Auxiliary theta: benchmark, sm
15:10    # samples: 1
15:10  xsec for this sampling theta: (0.2822914181342214 +/- 0.0027648833773815146) pb
15:10  Cumulative probability (should be close to 1): 1.0000058066463215
15:10  Cumulative x shape: (1905, 1, 6)
15:10  New theta configuration
15:10    Sampling theta: morphing, [-0.27598424  6.66538409]
15:10    Auxiliary theta: benchmark, sm
15:10    # samples: 1
15:10  xsec for this sampling theta: (0.2979987818467614 +/- 0.0018117736744

15:10    Sampling theta: morphing, [-4.42643676 -9.81320152]
15:10    Auxiliary theta: benchmark, sm
15:10    # samples: 1
15:10  xsec for this sampling theta: (0.3395962960238469 +/- 0.005669455718583322) pb
15:10  Cumulative probability (should be close to 1): 0.9999880860703378
15:10  Cumulative x shape: (1927, 1, 6)
15:10  New theta configuration
15:10    Sampling theta: morphing, [-8.72105636  3.14903965]
15:10    Auxiliary theta: benchmark, sm
15:10    # samples: 1
15:10  xsec for this sampling theta: (0.361556183414352 +/- 0.0033675045258549792) pb
15:10  Cumulative probability (should be close to 1): 1.0000053288355382
15:10  Cumulative x shape: (1928, 1, 6)
15:10  New theta configuration
15:10    Sampling theta: morphing, [4.43297646 5.25270989]
15:10    Auxiliary theta: benchmark, sm
15:10    # samples: 1
15:10  xsec for this sampling theta: (0.2681444209741639 +/- 0.01807943936050192) pb
15:10  Cumulative probability (should be close to 1): 1.000076610103534
15:10  Cumulativ

In [ ]:
x, theta = refinery.extract_samples_test(
    theta=constant_benchmark_theta('sm'),
    n_samples=10000,
    folder='./data/samples',
    filename='test'
)

Let's have a look at some distributions and correlations in this test sample:

In [ ]:
import corner

labels = [r'$p_{T,j1}$ [GeV]', r'$p_{T,j2}$ [GeV]', r'$\Delta \phi_{jj}$', r'$\Delta \eta_{jj}$', r'$m_{jj}$', r'$n_{j}$']
ranges = [(0., 300.), (0., 200.), (0.,6.2), (0.,8.), (0.,1500.), (1.,6.5)]

_ = corner.corner(x, color='C0', labels=labels, range=ranges)

For debugging, you can also access the full list of observables and benchmark weights (same units as in the LHE file) in the HDF5 file:

In [ ]:
all_x, all_weights = refinery.extract_raw_data(theta=[0.0,0.1])

print(all_x)
print(all_weights)

## 7. Train likelihood ratio estimator (e.g. RASCAL)

In [ ]:
train(
    'models/rascal.pt',
    method='rascal',
    theta_filename='data/samples/theta0_train_rascal.npy',
    x_filename='data/samples/x_train_rascal.npy',
    y_filename='data/samples/y_train_rascal.npy',
    r_xz_filename='data/samples/r_xz_train_rascal.npy',
    t_xz_filename='data/samples/t_xz_train_rascal.npy'
)